# Curve Decomposition

In [ ]:
from molass import get_version
assert get_version() >= '0.4.2', "This tutorial requires molass version 0.4.0 or higher."
from molass_data import get_version
assert get_version() >= '0.3.0', "This tutorial requires molass_data version 0.3.0 or higher."
from molass_data import SAMPLE4
from molass.DataObjects import SecSaxsData as SSD
ssd = SSD(SAMPLE4)
trimmed_ssd = ssd.trimmed_copy()
corrected_ssd = trimmed_ssd.corrected_copy()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
icurve  = corrected_ssd.xr.get_icurve()
from importlib import reload
import molass.Decompose.Proportional
reload(molass.Decompose.Proportional)
from molass.Decompose.Proportional import decompose_proportionally
x, y = icurve.get_xy()

result = decompose_proportionally(x, y, [3,2,1], debug=True)


In [ ]:
result

In [ ]:
len(x)

In [ ]:
rgcurve = corrected_ssd.xr.compute_rgcurve()

In [ ]:
rgcurve.indeces

In [ ]:
def plot_varied_proportions(num_components=3, rgcurve=None):
    from molass.SEC.Models.Simple import egh
    num_trials = 8
    trials = [f'Trial {i+1}' for i in np.arange(num_trials)]

    props_tuple = (3, 2, 1) if num_components == 3 else (3, 2)
    proportion_matrix = np.array([props_tuple]*num_trials, dtype=float)
    proportion_matrix[:,1] = np.linspace(1.0, 3.0, num_trials)
    proportions = []
    values = []
    results = []
    for props in proportion_matrix:
        props_ = props/np.sum(props)
        proportions.append(props_)
        result = decompose_proportionally(x, y, props_)
        values.append(result.fun)
        results.append(result)
    bottom = np.zeros(num_trials)

    fig1, axes = plt.subplots(nrows=2, ncols=4, figsize=(20, 8), sharex=True)

    for i, ax in enumerate(axes.flat):
        if i < num_trials:
            ax.plot(x, y, color='gray', alpha=0.5)
            cy_list = []
            for params in results[i].x.reshape((num_components, 4)):
                cy = egh(x, *params)
                ax.plot(x, cy, ':')
                cy_list.append(cy)
            ty = np.sum(cy_list, axis=0)
            ax.plot(x, ty, color='red', alpha=0.5)
            props_str = ", ".join(["%.3g" % p for p in proportion_matrix[i]])
            ax.set_title("%s - Proportions: [%s]" % (trials[i], props_str))
            ax.legend()
            if rgcurve is not None:
                axt = ax.twinx()
                axt.set_ylabel("$R_g$")
                cm = plt.get_cmap('YlGn')
                x_ = x[rgcurve.indeces]
                axt.grid(False)
                sc = axt.scatter(x_, rgcurve.rgvalues, c=rgcurve.scores, s=3, cmap=cm)

    fig2, ax = plt.subplots()
    ax.set_title("Variation of Proportion and Objective Function Value")
    for species, props_row in enumerate(np.array(proportions).T):
        # 
        ax.bar(trials, props_row, label=f'Species: {species+1}', bottom=bottom, alpha=0.3)
        bottom += props_row
    ax.legend(loc='lower left')
    ax.set_ylabel("Proportion")
    ax.set_xlabel("Trials")

    axt = ax.twinx()
    axt.plot(trials, values, 'o-', color='C3', label='Objective function value')
    axt.set_ylabel("Objective function value")
    axt.legend(loc='upper right')

In [ ]:
plot_varied_proportions(num_components=2, rgcurve=rgcurve)

In [ ]:
plot_varied_proportions(num_components=3, rgcurve=rgcurve)

In [ ]:
from molass_data import SAMPLE1
ssd = SSD(SAMPLE1)
trimmed_ssd = ssd.trimmed_copy()
corrected_ssd = trimmed_ssd.corrected_copy()

In [ ]:
modified_decomposition = corrected_ssd.quick_decomposition(num_components=3, proportions=[0.32, 0.20, 0.48])
plot2 = modified_decomposition.plot_components(title="Modified Decomposition of Sample1 (num_components=3, proportions=[0.32, 0.20, 0.48])")